<a href="https://colab.research.google.com/github/skj092/Apple-Orange-Detection/blob/main/Apple_Orange_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

In [ ]:
%%bash

kaggle datasets download -d mbkinaci/fruit-images-for-object-detection

unzip -q fruit-images-for-object-detection.zip && rm fruit-images-for-object-detection.zip

In [ ]:
# This script will more the images and labels into the correct folders as per YOLO format

import os
import shutil

# directory where the train data and test data are stored
INPUT_TRAIN_DIR = "train_zip/train"
INPUT_TEST_DIR = "test_zip/test"

# directory where the IMAGES AND LABELS are stored
OUTPUT_IMAGES_DIR = "images"
OUTPUT_LABELS_DIR = "labels"


# Create the folders for the images and labels
if not os.path.exists('images'):
    os.makedirs('images')
if not os.path.exists('labels'):
    os.makedirs('labels')

# Create the folders for the train and test images and labels
if not os.path.exists('images/train'):
    os.makedirs('images/train')
if not os.path.exists('images/test'):
    os.makedirs('images/test')

# Create the folders for the train and test labels
if not os.path.exists('labels/train'):
    os.makedirs('labels/train')
if not os.path.exists('labels/test'):
    os.makedirs('labels/test')



# Copy the images and labels into the correct folders
for filename in os.listdir(INPUT_TRAIN_DIR):
    if filename.endswith(".jpg"):
        shutil.copy(INPUT_TRAIN_DIR + "/" + filename, OUTPUT_IMAGES_DIR + "/train/" + filename)
    if filename.endswith(".xml"):
        shutil.copy(INPUT_TRAIN_DIR + "/" + filename, OUTPUT_LABELS_DIR + "/train/" + filename)

for filename in os.listdir(INPUT_TEST_DIR):
    if filename.endswith(".jpg"):
        shutil.copy(INPUT_TEST_DIR + "/" + filename, OUTPUT_IMAGES_DIR + "/test/" + filename)
    if filename.endswith(".xml"):
        shutil.copy(INPUT_TEST_DIR + "/" + filename, OUTPUT_LABELS_DIR + "/test/" + filename)


In [ ]:
# This script will convert xml files to YOLO format txt files

import os
import xml.etree.ElementTree as ET

# Path to train xml files
xml_path_train = "labels/train"
# Path to test xml files
xml_path_test = "labels/test"

# xml to txt
def xml_to_txt(xml_path):
    for xml_file in os.listdir(xml_path):
        if xml_file.endswith(".xml"):
            tree = ET.parse(xml_path + "/" + xml_file)
            root = tree.getroot()
            for member in root.findall('object'):
                filename = root.find('filename').text
                filename = filename[:-4]
                filename = filename + ".txt"
                x_min = int(member[4][0].text)
                y_min = int(member[4][1].text)
                x_max = int(member[4][2].text)
                y_max = int(member[4][3].text)
                width = x_max - x_min
                height = y_max - y_min
                x_center = x_min + width / 2
                y_center = y_min + height / 2
                with open(xml_path + "/" + filename, "a") as f:
                    f.write("0" + " " + str(x_center) + " " + str(y_center) + " " + str(width) + " " + str(height) + "")

xml_to_txt(xml_path_train)
xml_to_txt(xml_path_test)

In [ ]:
# YOLO 
!git clone https://github.com/ultralytics/yolov5  && cd yolov5 && pip install -r requirements.txt

In [ ]:
# check the assigned GPU type
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
    
else:
    print(gpu_info)

Fri Oct 28 07:41:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
yaml_text = """train: /content/images/train
val: /content/images/test

nc: 2
names: ['apple','orange']"""

In [ ]:
with open("wheat.yaml", 'w') as f:
    f.write(yaml_text)
%cat wheat.yaml

train: /content/images/train
val: /content/images/test

nc: 2
names: ['apple','orange']

In [ ]:
!python ./yolov5/train.py --img 512 --batch 2 --epochs 3 --workers 2 --data wheat.yaml --cfg "./yolov5/models/yolov5s.yaml" --name yolov5x_fruit --cache

train: weights=yolov5/yolov5s.pt, cfg=./yolov5/models/yolov5s.yaml, data=wheat.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=2, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=yolov5/runs/train, name=yolov5x_fruit, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-211-g32a9218 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t